In [4]:
import numpy as np
import json
import argparse
import os
import random
random.seed(2023)

import gensim.corpora as corpora
from tqdm import tqdm
from model import ThetaRoleModel

# prepare the corpus:

In [3]:
from malt import MaltParser # source code from nltk library

In [ ]:
# ans = process_dataset(sample_corpus,stop_words,prep_words)

In [46]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from tqdm import tqdm
import json
#from datasets import load_dataset
from gensim import utils
from collections import defaultdict

import re

from word import Word
# from malt.malt import MaltParser # source code from nltk library
from malt import MaltParser # source code from nltk library

def process_dataset(docs, stop_words, pre_words, malt_parser_version='../maltparser-1.7.2', model_version='engmalt.linear-1.7.mco'):
    # initalize malt parser model
    mp = MaltParser(malt_parser_version, model_version, tagger=nltk.pos_tag)
    
    ids = list(docs.id)
    docs = list(docs.document)

    #get the documents as a list BUT ALSO NEED TO FIND A WAY TO KEEP THE IDS
    #LIKE WE NEED TO PREPROCESS AND PUT THEM BACK TO DOCS[1]
    #Add more preprocessing:
    # replace single smart quote with single straight quote, so as to catch stopword contractions
    docs = [re.sub("[\u2018\u2019]", "'", doc) for doc in docs] #replace qoute with regualar qoutations
    #it removes the digits
    # docs = [re.sub('\d+', '', doc) for doc in docs] 
    docs = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in docs]
    docs = [re.sub(r"http\S+", '', doc) for doc in docs]
    
    # create <doc_idx, tokenized_sent> list of sents
#     sents = [
#         (i, j[0], nltk.word_tokenize(sent))
#         for i, doc in enumerate(docs)
#         for sent in nltk.sent_tokenize(utils.to_unicode(str(doc).lower())) # convert doc to lowercase, and sentence tokenized.
#         for s, j in enumerate(docs)
#     ]
    
    sents = [
            (ids[i], nltk.word_tokenize(sent))
            for i, doc in enumerate(docs)
            for sent in nltk.sent_tokenize(utils.to_unicode(str(doc).lower())) # convert doc to lowercase, and sentence tokenized.
#             for s, j in enumerate(docs)
        ]

    # unzip list of tuples
    doc_idxs, sents = zip(*sents)
    print('----------- *********** --------------')
#     print(doc_idxs, sents)

    # create parser <generator> and loop through parser to produce dependency tree for each sentence
    parser = mp.parse_sents(sents, verbose=True)

    # define valid word
    stop_words.append('amp');stop_words.append('&amp');stop_words.append('&amp;')
    valid_word = lambda word: not word in stop_words and word.isalpha() and len(word) > 2

    # initalize dictionary for json output
    docs_dict = {
        #'documents': dict((doc_idx, {'words': [], 'relns': [], 'arg2':[] , 'originaltext': []}) for doc_idx in doc_idxs),
        'documents': dict((doc_idx, {'words': [], 'relns': [], 'arg2':[] , 'originaltext': []}) for doc_idx in doc_idxs),
        'vocab': [], 
        'vocab_relns': [],
        'vocab_arg':[],
    }
    
    # initalize vocab variables as sets (no duplicates)
    vocab = set()
    vocab_relns = set()
    vocab_arg = set()
    vocab_arg_test, vocab_relns_test = set(), set()

    i = 0
    # loop through list iterators
    for list_it in parser:
        tree = next(list_it)
        # check if valid tree, if not skip
        try:
            nodes = tree.nodes
        except:
            continue
        #print(nodes)
#         print('******000000*****')
        word_relns_hash = defaultdict(list)
        word_args_hash = defaultdict(list)
        for word_idx in nodes:
            if word_idx == 0: # skip first
                continue

            deps = nodes[word_idx]['deps']
            
            if deps:
                for reln, idxs in deps.items():
                    for idx in idxs:
                        if reln =='prep':
                            if nodes[idx]['word'] in prep_words:
                                reln = 'prep'+'.'+nodes[idx]['word']
                                try:
                                    if len(list(nodes[idx]['deps'].values()))==0:
                                        continue
                                    elif type(list(nodes[idx]['deps'].values())[0])==list:
                                        idx = list(nodes[idx]['deps'].values())[0][0]
                                        if (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
                                            continue
                                    else:
                                        idx = list(nodes[idx]['deps'].values())[0]
                                        if (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
                                            continue
                                except:
                                    print('Error')
                                    return (nodes, idx)
                            
                            else:
                                continue
                            
                        
                        elif (not valid_word(nodes[word_idx]['word']) or (not valid_word(nodes[idx]['word']))):
                            continue

                        dep_reln, gov_reln = f"{reln}.dep", f"{reln}.gov"
                        if gov_reln =="punct.gov":
                            continue
                        
                        
                        # add reln to word in hashmap
                        word_relns_hash[idx].append(dep_reln) # append to dep word
                        word_relns_hash[word_idx].append(gov_reln) # append to current word
                        word_args_hash[idx].append(nodes[word_idx]['word'])

                        #TESTING UNCOMENTING IT ONCE AND SEE IF IT FIX THE ISSUE... 
                        word_args_hash[word_idx].append(nodes[idx]['word'])
                        
                        
                        
        
        # check for valid hashmap
        if word_relns_hash:
            doc_idx = doc_idxs[i]
            # loop through hashmap items and append to dict for future storing
            for word_idx, relns in word_relns_hash.items():
                word = nodes[word_idx]['word']
                if valid_word(word):
                    vocab.add(word)
                    
                    docs_dict['documents'][doc_idx]['words'].append(word)
                    docs_dict['documents'][doc_idx]['relns'].append(relns)
                    index_id = ids.index(doc_idx)
                    try:
                        docs_dict['documents'][doc_idx]['originaltext'] = [docs[index_id]]
                    except:
                        print(index_id, doc_idx, docs_dict['documents'])
                        return 0
                    #docs_dict['documents'][doc_idx]['originaltext'] = [docs[doc_idx]]
                    docs_dict['documents'][doc_idx]['arg2'].append(word_args_hash[word_idx])
                    try:
                        vocab_arg.update(word_args_hash[word_idx])
                        vocab_relns.update(relns)
                    except:
                        print(word_args_hash[word_idx])
#                         print(vocab_arg_test)
                        return 0
        
        i += 1

    docs_dict['vocab'] = list(vocab)
    docs_dict['vocab_relns'] = list(vocab_relns)
    docs_dict['vocab_arg'] = list(set(vocab_arg))

    return docs_dict

# papare the data for theta role model:

In [5]:
# set the stop words
import pickle
# with open("../Stopword_list",'rb') as read_file:
with open('extra_stopwords','rb') as read_file:
    more_stop_words = pickle.load(read_file)
# stop word initialization
# with open("data/utils/stopwords.txt") as f:
#     more_stop_words = f.read().splitlines()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(more_stop_words)

In [ ]:
# 'predominante' in stop_words, 'persons' in stop_words

In [6]:
type(stop_words)
json_stopwords = json.dumps(stop_words)
with open('all_stopwrods.json', "w") as f:
    f.write(json_stopwords)

In [8]:
import mysql.connector

sql_db = mysql.connector.connect(host='127.0.0.1',user = 'covidAnalysis',password = 'k34p63MbDDcZ9yf4',
                                 database = 'covid19framing')
cursor = sql_db.cursor()

In [9]:
cursor.execute('show tables;')
results = cursor.fetchall()


In [10]:
cursor.execute('show tables;')
results = cursor.fetchall()
# I changed the chosen column to be text_preproc and not text because of some issues in those rows wherein text_preproc is null and text exist...
cursor.execute("select * from articles where text_preproc is not NULL")
results = cursor.fetchall()
len(results)

3699

In [11]:
originaltext = [item[1].decode() for item in results] #originaltextoriginaltext[1]
doc_ids = [item[0] for item in results]
originaltext_do_ids ={doc_ids[i]: originaltext[i] for i in range(len(originaltext))}
# corpus = pd.Series(originaltext)
#Here we need to sample documents with id and then store the id into the preprocessed as well... 
corpus = originaltext_do_ids.items()
corpus = pd.DataFrame(corpus,columns=['id','document'])
sample_size = int(len(corpus) * 1)
sample_corpus = corpus.sample(n=sample_size)
len(sample_corpus)

3699

In [25]:
# 

In [24]:
type(corpus[corpus.id==2980])

pandas.core.frame.DataFrame

In [22]:
import pickle
with open('prep_words.txt', "r") as file:
    prep_words = [line.strip() for line in file]
# print(prep_words)

## testign blocks ..

In [37]:
docs_dict = process_dataset(corpus[corpus.id==2980], stop_words, prep_words)

----------- *********** --------------


-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Mon Nov 20 14:54:02 EST 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    358MB
.          	     10	      4s	    380MB
           	     13	      4s	    380MB
Parsing time: 00:00:03 (3516 ms)
Finished: Mon Nov 20 14:54:07 EST 2023


In [45]:
docs_dict['documents'][2980]['words'][0], docs_dict['documents'][2980]['relns'][0],  docs_dict['documents'][2980]['arg2'][0]

('data',
 ['nn.dep',
  ['prep.of.dep', 'amod.gov', 'prep.on.gov'],
  ['dobj.dep', 'rcmod.gov'],
  ['prep.in.gov', 'nsubj.dep'],
  ['prep.for.gov', 'nsubjpass.dep'],
  ['nn.dep'],
  ['dobj.dep', 'amod.gov'],
  ['prep.on.gov'],
  ['pobj.dep'],
  ['nn.gov'],
  ['nn.dep'],
  ['pobj.dep'],
  ['dobj.dep', 'partmod.gov'],
  ['nsubjpass.dep'],
  ['nn.dep']],
 ['updates'])

In [34]:
docs_dict = process_dataset(corpus[corpus.id==2980], stop_words, prep_words)

----------- *********** --------------


-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Mon Nov 20 12:30:00 EST 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      3s	    364MB
.          	     10	      4s	    386MB
           	     13	      4s	    386MB
Parsing time: 00:00:03 (3101 ms)
Finished: Mon Nov 20 12:30:04 EST 2023


In [ ]:
docs_dict

In [28]:
docs_dict

{'documents': {2980: {'words': ['data',
    'updates',
    'testing',
    'state',
    'continues',
    'taking',
    'connecticut',
    'actions',
    'response',
    'provided',
    'global',
    'spread',
    'disease',
    'governor',
    'ned',
    'lamont',
    'updates',
    'coronavirus',
    'following',
    'summary',
    'data',
    'tests',
    'connecticutthe',
    'following',
    'reported',
    'cases',
    'newly',
    'deaths',
    'connecticut',
    'note',
    'important',
    'include',
    'updates',
    'data',
    'reported',
    'newly',
    'occurred',
    'days',
    'last',
    'several',
    'report',
    'data',
    'preliminary',
    'updated',
    'dates',
    'data',
    'received',
    'previous',
    'reports',
    'summarytotalchange',
    'new',
    'data',
    'errors',
    'reported',
    'since',
    'cases',
    'tests',
    'confirmed',
    'molecular',
    'probable',
    'positivity',
    'patients',
    'antigen',
    'test',
    'hospitaliz

In [12]:
docs_dict = process_dataset(sample_corpus, stop_words, prep_words)

----------- *********** --------------


-----------------------------------------------------------------------------
                          MaltParser 1.7.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Fri Nov 03 10:54:37 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    364MB
.          	     10	      5s	    375MB
.          	    100	      5s	    441MB
..........	   1000	     11s	    781MB
..........	   2000	     17s	    541MB
..........	   3000	     20s

In [ ]:
docs_dict

# Store the parsed data to DB to reference to and extract examples 

In [12]:
# df = pd.DataFrame(columns =['document_id', 'word', 'relns', 'arg2'] )
# len(docs_dict)

In [29]:
# ids = list(docs_dict['documents'].keys())
# # len(ids), type(ids), list(ids)
# len(docs_dict['documents'][5267]['words']), len(docs_dict['documents'][5267]['relns']), len(docs_dict['documents'][5267]['arg2'])

(156, 156, 156)

In [33]:
# ','.join(docs_dict['documents'][5267]['relns'][2]).split(',')

['parataxis.dep', 'parataxis.gov', 'dobj.gov', 'xcomp.gov']

In [38]:

# docs_dict['documents'][5267]['relns'][2]


['parataxis.dep', 'parataxis.gov', 'dobj.gov', 'xcomp.gov']

In [34]:
# ','.join(docs_dict['documents'][5267]['arg2'][2]).split(',')

['may', 'contact', 'pantries', 'taking']

In [37]:
# docs_dict['documents'][5267]['words'][2]

'provided'

## Store All the parsed document in a dataframe

In [13]:
# DO NOT RUN THIS BLOCK; it is to store parsedDoh documents
from tqdm import tqdm
ids = list(docs_dict['documents'].keys())
ls = []

for doc_id in tqdm(ids):
    doc_word_count = len(docs_dict['documents'][doc_id]['words'])
    for widx in range(doc_word_count):
        ls.append({'document_id':doc_id,'word':docs_dict['documents'][doc_id]['words'][widx],\
                  'relns':','.join(docs_dict['documents'][doc_id]['relns'][widx]),\
                  'arg2':','.join(docs_dict['documents'][doc_id]['arg2'][widx])})
df = pd.DataFrame(ls)
del ls

100%|██████████| 2774/2774 [00:00<00:00, 4994.01it/s]


In [14]:
df.to_csv('parsedDoh_three_quarter.csv')

In [15]:
docs_dict['documents'][5267]['originaltext']

["FOR IMMEDIATE RELEASE:\xa0May 1, 2020\xa0CONTACT:\xa0Bob Wheaton, 517-241-2112LANSING, Mich. – The Michigan Department of Health and Human Services (MDHHS) will continue to provide nutritious food by the truckload to Flint residents throughout May.MDHHS has provided the Flint mobile food pantries since February 2016 in partnership with the Food Bank of Eastern Michigan in Flint.Since the outbreak of COVID-19 in Michigan, food pantry distributors are taking extra precautions to keep customers and volunteers protected. Each site is requiring volunteers to wear gloves and masks, clean their hands with hand sanitizer often, and take steps to sanitize distribution buildings throughout the day. Customers do not have to step out of their vehicle to receive food – volunteers are asking the customer to simply open their trunk and they will load up the vehicle with food to reduce contact.This month's mobile food pantry distribution will have plenty of delicious, vitamin-rich produce, including

In [48]:
df

,document_id,word,relns,arg2
0,5267,immediate,amod.dep,release
1,5267,release,amod.gov,immediate
2,5267,provided,"parataxis.dep,parataxis.gov,dobj.gov,xcomp.gov","may,contact,pantries,taking"
3,5267,may,parataxis.gov,provided
4,5267,contact,parataxis.dep,provided
...,...,...,...,...
811586,2828,strongly,advmod.dep,urged
811587,2828,contact,"xcomp.dep,dobj.gov","urged,provider"
811588,2828,symptoms,dobj.dep,experiencing
811589,2828,provider,"dobj.dep,amod.gov","contact,medical"


In [ ]:
# len(docs_dict['vocab_arg']), len(docs_dict['vocab_arg_test'])
# len(docs_dict['vocab_relns']), len(docs_dict['vocab_relns_test'])
# set(docs_dict['vocab_relns']).difference(set(docs_dict['vocab_relns_test']))

In [16]:
len(docs_dict['documents'])

2774

In [17]:
json_object = json.dumps(docs_dict, indent=4)
# with open("sample_10_withoutStopPrepNotCombined.json", "w") as f:
with open("threeQuarter_DoH.json", "w") as f:
    f.write(json_object)

# Read the test dataset

In [18]:
#Choose the data and the subsample 10 % of the data to run the model on:
with open("threeQuarter_DoH.json") as json_file:
# with open("doh_corpus_5_sample.json") as json_file:
    o = json.load(json_file)
len(o['documents'])

2774

In [25]:
# o['documents']['4224']

In [4]:
len(o['documents']), len(o['vocab']), len(o['vocab_relns'])

(184, 6467, 176)

In [16]:
# with open("sample_10_without_stopwords.json") as json_file:
with open("sample_15_prepCollapsed.json") as json_file:
    o = json.load(json_file)
doc_objects = o['documents'] #words, reln, originaltext
docs = [ doc_objects[str(doc_id)]['words'] for doc_id in doc_objects ]
text = [ doc_objects[str(doc_id)]['originaltext'] for doc_id in doc_objects ]
doc_relns = [ doc_objects[str(doc_id)]['relns'] for doc_id in doc_objects ]
doc_arg2 = [ doc_objects[str(doc_id)]['arg2'] for doc_id in doc_objects ]
vocab = o['vocab']
vocab_relns = o['vocab_relns']
vocab_arg = o['vocab_arg']

In [27]:
'the' in vocab_arg, 'and' in vocab_arg, 'prep.in.gov' in  vocab_relns, 'prep.gov' in vocab_relns

(False, False, True, False)

In [ ]:
# 'prep.persons.for' in vocab_relns
# 'the' in vocab_arg

In [17]:
# document preprocessing helpers
id2word = corpora.Dictionary(docs)
reln2id = {reln:i for i, reln in enumerate(vocab_relns)}
arg2id = {arg:i for i, arg in enumerate(vocab_arg)}
corpus = list(map(lambda x: id2word.doc2idx(x), docs))
# originaltext = list(map(lambda x: id2word.doc2idx(x), text))
#originaltext = list(map(lambda doc_id: doc_objects[str(doc_id)]['originaltext'], doc_objects))
originaltext = {doc_id: doc_objects[str(doc_id)]['originaltext'] for doc_id in doc_objects}
idx2docid =  {i:doc_id for i, doc_id in enumerate(doc_objects)}

# initialize scalars from plate diagram
D, V, R, A2= len(docs), len(vocab), len(vocab_relns), len(vocab_arg) # n documents, n words, n relns: https://universaldependencies.org/u/dep/

# initialize theta role model

In [18]:
#set the arguments.
K, T= 5, 10
alpha, eta, etaprime, gamma, lam , omega= .1, .1, .1, .1, 0.1, 0.1
n_iters = 1000
corpusName = 'sample_15_prepCollapsed'

In [3]:
'sample_15_prepCollapsed.json'.split('json')[0][0:-1]

'sample_15_prepCollapsed'

In [19]:
theta_model = ThetaRoleModel(corpus, originaltext, doc_relns, doc_arg2, vocab_relns, vocab_arg, id2word, reln2id, arg2id, idx2docid, n_iters, K, T, D, V, R, A2, alpha, eta, etaprime, gamma, lam, omega, corpusName)
theta_model.initialize_variables()

In [ ]:
# 'nsubjpass.dep' in reln2id, 'nsubjpass.dep' in vocab_relns, 'nsubjpass.dep' in doc_relns
# len(vocab_relns)

In [ ]:
# 'county4middlesex' in arg2id, 'county4middlesex' in vocab_arg, 
# 'county4middlesex' in arg2id, 'county4middlesex' in vocab_arg,'county4middlesex' in doc_arg2

In [21]:
# doc_relns

In [20]:
theta_model.fit()

# compute matrices
theta_model.compute_matrices()
# print topics, theta roles, and top topics/theta roles for each document
theta_model.print_all()


100%|██████████| 1000/1000 [14:28:18<00:00, 52.10s/it] 


TOPICS TOP TERMS, TOP GRAMMATICAL RELN, and TOP ARG2 distributions ****************** 
 
Topic 0: 0.016447360823361214 * state, 0.013144492409176894 * governor, 0.008963938682132407 * order, 0.007462634857503171 * emergency, 0.0066311435084777486 * health, 0.006469464635056138 * support, 0.006446367653138765 * department, 0.006146106888212918 * connecticut, 0.005984428014791308 * program, 0.005845846123287071 * services, 0.005776555177534952 * care, 0.00568416724986546 * assistance, 0.0055917793221959685 * today, 0.005568682340278596 * public, 0.005245324593435376 * businesses, 0.004945063828509529 * pandemic, 0.004714094009335801 * federal, 0.004667900045501054 * help, 0.004598609099748936 * food, 0.0044600272082446985 * provide 

 *** Top theta roles 4 - 0.927190174739787 , for reln component ** 

Distribution of reln for each words of the topic 0
* Relns nn.dep: 0.10748897479174606, nn.gov: 0.09948193948977489, amod.gov: 0.08053846572657485, amod.dep: 0.07669177470936771, dobj.dep: 

In [10]:
# def print_top_documents_topic(self, top_doc=10):
top_doc=10
'''
print top documents for each topic
print only 10 top document, but store in db top 50 
'''
import pandas as pd
df = pd.DataFrame(columns =['topic_id', 'document_ids', 'probability_score'])
ls = []
for k in range(theta_model.K):
    top_documents_indx = np.argsort(theta_model.theta[:, k])[::-1][:]
    #top_documents_indx = np.argsort(self.theta[:, k])[::-1][:top_doc]
    top_thetas = 10
    print('topic {0} \n'.format(k))
    try:
        c=0
        for idx in top_documents_indx:
            if c<=top_doc:
                print("doc {0} - {1} - {2} \n ".format(idx, theta_model.theta[idx, k], theta_model.originaltext[theta_model.idx2docid(idx)]))
            ls.append({'topic_id':k,'document_ids':idx,\
                  'probability_score':theta_model.theta[idx, k]})
            c+=1
        print('------------------------------------------------------------------------')
    except:
        print('An error occured here in TOP DOCUMENTS.......')
#                 return top_documents_indx
        np.argsort(theta_model.theta[:, k])[::-1]
#store probability of all documents for each topic in a dataframe and in csv file.
df = pd.DataFrame(ls)
# df.to_csv('topicsDocProbabilities.csv')


topic 0 

An error occured here in TOP DOCUMENTS.......
topic 1 

An error occured here in TOP DOCUMENTS.......
topic 2 

An error occured here in TOP DOCUMENTS.......
topic 3 

An error occured here in TOP DOCUMENTS.......
topic 4 

An error occured here in TOP DOCUMENTS.......


In [ ]:
# fix the issue with vocab reln
# output = []
# def reemovNestings(l):
#     for i in l:
#         if type(i) == list:
#             reemovNestings(i)
#         else:
#             output.append(i)
    
# reemovNestings(doc_relns)
# # output
# unique_reln = set(output)
# vocab_relns = [item for item in vocab_relns if item in unique_reln]

In [13]:
theta_model.idx2docid[idx]

'4251'

In [15]:
for idx in top_documents_indx:
    if c<=top_doc:
        print("doc {0} - {1} - {2} \n ".format(idx, theta_model.theta[idx, k], theta_model.originaltext[str(idx)]))
    ls.append({'topic_id':k,'document_ids':idx,\
          'probability_score':theta_model.theta[idx, k]})
    c+=1

KeyError: '147'

In [26]:
theta_model.originaltext[str(idx)]
# So some texts are null that's why it happens. I need to go back and check the pre-processing  

KeyError: '147'

In [ ]:
#to test relns that have zero propbabilites:
for reln in theta_model.vocab_relns:
    temp = np.sum(theta_model.n_w_reln[:, theta_model.reln2id[reln]])
    if temp ==0:
        print(reln)

In [ ]:
for reln in theta_model.vocab_relns:
    temp = np.sum(theta_model.n_w_reln[:, theta_model.reln2id[reln]])
    if temp ==0:
        print(reln)

In [ ]:
for arg2 in theta_model.vocab_args:
    temp = np.sum(theta_model.n_w_arg2[:, theta_model.arg2id[arg2]])
    if temp ==0:
        print(arg2)

In [ ]:
# doc_relns.index('rel.gov')
vocab_relns.index('possessive.dep')

In [ ]:
(theta_model.n_t_arg2[:, theta_model.arg2id['what']] + theta_model.etaprime) \
                                     / (np.sum(theta_model.n_t_arg2[:, :],axis=1) + theta_model.etaprime * theta_model.A2)

In [ ]:
np.sum(theta_model.betaprime[0,:]), np.sum(theta_model.zeta[0,:])

In [ ]:
theta_model.A2, theta_model.V, 

In [ ]:
p_t1 =np.asarray([0.00419139, 0.00328302 ,0.00330478, 0.00368039 ,0.00334062 ,0.00311779
 ,0.00439059, 0.00251055 ,0.00303268 ,0.00305069])
p_t1 / sum(p_t1)

In [ ]:
# def count_arg(arg):
#     c=0
#     for ls in theta_model.doc_arg2:
#         for ls2 in ls:
#             c+=ls2.count(arg)
#     return c

# widx, t =2,2

# res = {}
# for arg in theta_model.vocab_args:
#     #p(w|arg2) : p_w_arg
#     p_w_arg= theta_model.n_w_arg2[widx, theta_model.arg2id[arg]]/count_arg(arg)
#     p_arg_theta = theta_model.betaprime[t, theta_model.arg2id[arg]]
#     res[arg] = p_w_arg*p_arg_theta
# print(res)

In [ ]:
arg

# The rest is testing blokcs 

In [ ]:
theta_model.n_t_k[9,:]
# , np.sum(theta_model.n_t_k[:, 0]/np.sum(theta_model.n_t_k[:, 0])), theta_model.n_t_k[:, 0]/np.sum(theta_model.n_t_k[:, 0])

In [ ]:
# theta_model.top_theta_role_for_topic(0, 3)
theta_model.phi[0,:]

In [ ]:
np.sum(theta_model.n_k_w[0]), np.sum(theta_model.n_k[0])

In [ ]:
np.sum(theta_model.n_t_w[0]), np.sum(theta_model.n_t[0]), theta_model.n_t_w[0]

In [ ]:
np.sum(theta_model.n_t_reln[0]), theta_model.n_t[1], np.sum(theta_model.n_t_reln)

In [ ]:
np.sum(theta_model.betaprime, axis = 1)[0], np.sum(theta_model.zeta, axis = 1)[0]

In [ ]:
np.argsort(theta_model.phi[0, :])[::-1][:2]

In [ ]:
np.sum(theta_model.betaprime[5,:])

In [ ]:
theta_model.phi = np.zeros((theta_model.K, theta_model.T))
for k in range(theta_model.K):
    for t in range(theta_model.T):
        theta_model.phi[k, t] = (theta_model.n_t_k[t, k] + theta_model.gamma) / (np.sum(theta_model.n_t_k[:, k]) + theta_model.T * theta_model.gamma)  #@@@@CHECK THIS ONE
theta_model.phi

In [ ]:
theta_model.phi[k, t]

In [ ]:
# theta_model.betaprime[0].shape, len(theta_model.arg2id)
np.argsort(theta_model.betaprime[1])[::-1][:10]
# theta_model.betaprime[0, 64]
np.argsort(theta_model.betaprime[1])[::-1][:10], theta_model.betaprime[1, 394], np.sort(theta_model.betaprime[1, :])[::-1]

In [ ]:
np.sort(theta_model.betaprime[1, :])[::-1], np.argsort(theta_model.betaprime[1, :])[::-1], theta_model.betaprime[1, 398]

In [ ]:
np.sort(theta_model.betaprime[1, :])[::-1], theta_model.betaprime[1, np.argsort(theta_model.betaprime[1, :])[::-1][0]]

In [ ]:
for t in range(10):
    args_ids = np.argsort(theta_model.betaprime[t])[::-1]
    probs = np.sort(theta_model.betaprime[t])[::-1]
    top_args = [theta_model.vocab_args[i] for i in args_ids]
    strings = [f'{prob} * {arg}' for prob, arg in zip(probs, top_args)]
    print("theta role {0}, {1} \n ".format(t, args_ids))
    #print(f"Theta Role \n {str(t)}: {', '.join(strings)}\n")

In [ ]:
# theta_model.phi = np.zeros((theta_model.T, theta_model.K))
# for k in range(theta_model.K):
#     theta_model.phi[k] = (theta_model.n_t_k[k] + theta_model.gamma) / (theta_model.n_k[k] + theta_model.T * theta_model.gamma)  #@@@@CHECK THIS ONE

In [ ]:
(theta_model.n_t_k[4] + theta_model.gamma) / (theta_model.n_t+ theta_model.T * theta_model.gamma)

In [ ]:
theta_model.phi[1]
(theta_model.n_t_k[1])/ (theta_model.n_t[1]), 
# + self.gamma) / (self.n_k[k] + self.T * self.gamma) 
tt = np.zeros((theta_model.T, theta_model.K))
for k in range(theta_model.K):
    tt[k] = (theta_model.n_t_k[k] + theta_model.gamma) / (theta_model.n_t[k] + theta_model.T * theta_model.gamma) 
tt

In [ ]:
[(theta_model.vocab_args[j], theta_model.betaprime[3,j]) for j in range(len(theta_model.betaprime[3,:]))]

In [ ]:
def top_theta_role_for_topic(k, tops): #k is topic id 
    top_theta_indx = np.argsort(theta_model.phi[k, :])[::-1][:tops]
    return top_theta_indx

In [ ]:
# results.items()

In [ ]:
# theta_model.compute_matrices()

# for t in range(theta_model.T):
#     args_ids = np.argsort(theta_model.betaprime[t])[::-1][:10]
#     probs = np.sort(theta_model.betaprime[t])[::-1][:10]
#     top_args = [theta_model.vocab_args[i] for i in args_ids]
#     strings = [f'{prob} * {arg}' for prob, arg in zip(probs, top_args)]
#     print(f"Theta Role \n {str(t)}: {', '.join(strings)}\n")

In [ ]:

# '''
# print topics word, and their reln distribution
# '''
# top_doc = 10
# tops = 1
# for k in range(theta_model.K):
#     word_ids = np.argsort(theta_model.beta[k])[::-1][:20]
#     probs = np.sort(theta_model.beta[k])[::-1][:20]
#     top_words = [theta_model.id2word[i] for i in word_ids]
#     strings = [f'{prob} * {word}' for prob, word in zip(probs, top_words)]
#     print(f"Topic {str(k)}: {', '.join(strings)} \n") 

#     #get the distribution over typed depdencies (relns) for each topic's terms..
#     for widx in word_ids:
#         #Add a distribution of p(reln|word, top theta role for that the given word)
#         #First, need to know top theta role for this specific topic based on distribution of each theta role for top documents...

#         top_theta_ids = theta_model.top_theta_role_for_wordidx(widx, tops)
#         print(' * {0}: top theta roles {1} - {2} \n'.format(str(theta_model.id2word[widx]), top_theta_ids[0], theta_model.betaprime[top_theta_ids[0], widx]))
#         dic_reln = {reln:0 for reln in theta_model.vocab_relns} 
#         for t in top_theta_ids:
#             dist = theta_model.compute_reln_distribution_for_wordidx_in_top_theta_t(widx, t)#@zhila double check this one.. 
#             #merge dist for all the top theta role for this word..sum dist or average dist
#             for ki,v in dist.items():
#                 dic_reln[ki] += v
#         #sort based on probability of reln for all the theta role...
#         results = sorted(dic_reln.items(), key=lambda x: x[1], reverse=True)
#         print(f"* {str(theta_model.id2word[widx])}: Relns: {', '.join([result[0]+':'+str(result[1]) for result in results[0:5]])} \n")
#         print('-------- -------- -------- -------- -------- --------')

# The rest of this notebook is testing blocks:

In [ ]:
#self.betaprime = np.zeros((self.T, self.V))
# def top_theta_role_for_wordidx(widx, tops):
#     '''
#     get the top theta role t for this specic word with id widx
#     '''
#     top_theta_indx = np.argsort(theta_model.betaprime[:, widx])[::-1][:tops]
#     return top_theta_indx
    
# def compute_reln_distribution_for_wordidx_in_top_theta_t(widx, thetaroleidx):
#     '''
#     get the p(reln|word within the top theta role thetat)
#     '''
#     res = {}
#     for i in range(len(theta_model.vocab_relns)):###
#         #p(reln, theta_t) SHOULD THIS BE FIND BASED ON ZETA MATRIX INSTEAD???
#         p_reln_y =(theta_model.n_t_reln[thetaroleidx, theta_model.reln2id[theta_model.vocab_relns[i]]] + theta_model.lam) \
#         / (theta_model.n_t[thetaroleidx] + theta_model.lam * theta_model.R)
#         #p(thetaroleidx, word_indx) *********** <<<<CHECK THIS WITH AMIN>>>>
#         p_t_w = (theta_model.betaprime[thetaroleidx, widx]) #I AM NOT SURE IF IT IS CORRECT....
#         p_relni_widx_thetaroleidx = p_reln_y * p_t_w
#         res[theta_model.vocab_relns[i]] = p_relni_widx_thetaroleidx

#     return res  

# top_doc = 10
# tops = 1
# for k in range(theta_model.K):
#     word_ids = np.argsort(theta_model.beta[k])[::-1][:20]
#     probs = np.sort(theta_model.beta[k])[::-1][:20]
#     top_words = [theta_model.id2word[i] for i in word_ids]
#     strings = [f'{prob} * {word}' for prob, word in zip(probs, top_words)]
#     print(f"Topic {str(k)}: {', '.join(strings)} \n") 

#     for widx in word_ids:
#         #Add a distribution of p(reln|word, top theta role for that the given word)
#         #First, need to know top theta role for this specific topic based on distribution of each theta role for top documents...

#         top_theta_ids = theta_model.top_theta_role_for_wordidx(widx, tops)
#         print(' * Top theta role for {0}: {1} \n'.format(str(theta_model.id2word[widx]), top_theta_ids))
#         dic_reln = {reln:0 for reln in theta_model.vocab_relns} #@zhila double check this one.. 
#         for t in top_theta_ids:
#             dist = theta_model.compute_reln_distribution_for_wordidx_in_top_theta_t(widx, t)
#             #merge dist for all the top theta role for this word..sum dist or average dist
#             for ki,v in dist.items():
#                 dic_reln[ki] += v
#         #sort based on probability of reln for all the theta role...
#         results = sorted(dic_reln.items(), key=lambda x: x[1], reverse=True)
#         print('-------- -------- -------- -------- -------- --------')
#         print(f"* {str(theta_model.id2word[widx])}: Relns: {', '.join([result[0]+':'+str(result[1]) for result in results[0:3]])} \n")

In [ ]:
def cfs(w):
    c = 0
    for sublist in corpus:
        c+= sublist.count(w)
    return c

In [ ]:
# len(corpus)

In [ ]:
def top_theta_role_for_topic(k, top_documents_indx): #k is topic id 
    #top doc index
#     top_doc = 5
#     top_documents_indx = np.argsort(theta_model.theta[:, k])[::-1][:top_doc]
    #top_thetas_for_doc_indx = []
    theta_dist = np.zeros((1,theta_model.phi.shape[1]))
    #for idx in top_documents_indx:
    #    theta_dist += (self.phi[idx])
    theta_dist = np.mean(theta_model.phi[top_documents_indx,:],axis=0)  
    #print('***')
    top_theta_ids = np.argsort(theta_dist)[::-1]
    #theta_dist = np.sum(theta_dist, axis=1)
    #return top_theta_ids
    return top_theta_ids

In [ ]:
#they should be chosen for words within each topic... 
def compute_reln_distribution_for_topic_words(widx, topicidx, thetaroleidx):
    res = {}
    #len(set(reln2id)) is 98!
    #p_reln = [0]*98
    #get the top reln for this theta role
#     topn_reln = 3
    reln_ids = np.argsort(theta_model.zeta[thetaroleidx])[::-1][:]
    #probs = np.sort(self.zeta[t])[::-1][:]
    top_relns = [theta_model.vocab_relns[i] for i in reln_ids] #name of these reln ids
    #compute p(reln|w_indx, topic_k, theta_t)
#     for i in reln_ids:
    for i in range(len(theta_model.vocab_relns)):######AZ
        #p(reln, theta_t)
        p_reln_y =(theta_model.n_t_reln[thetaroleidx, theta_model.reln2id[theta_model.vocab_relns[i]]] + theta_model.lam) \
        / (theta_model.n_t[thetaroleidx] + theta_model.lam * theta_model.R)
        #p(theta_y, topic_k)
        p_y_z = (theta_model.n_t_k.T[topicidx, thetaroleidx] + theta_model.gamma) \
        / (theta_model.n_k[topicidx] + theta_model.gamma * theta_model.K)
        #p(topic_k, word_indx)
        p_z_w = (theta_model.n_k_w.T[widx, topicidx] + theta_model.eta) / (cfs(widx) + theta_model.eta * theta_model.V)
        p_reln_w_z_y = p_reln_y * p_y_z * p_z_w
        #need to construct reln:prob here and return it as a string of reln distributions... maybe a tuple or dic so we can sort it based on probability scores?
        res[theta_model.vocab_relns[i]] = p_reln_w_z_y
        #res[i] = p_reln_w_z_y
    #sort this dicionary based on values and return the tuple of key values ? 
#         results = sorted(res.items(), key=lambda x: x[1], reverse=True)
    return res

In [ ]:
# def print_topics_word_reln_distribution():
'''
Print topics with words and each words' reln distribution
'''
theta_model.K = 10
top_doc = 10
for k in range(1):
    word_ids = np.argsort(theta_model.beta[k])[::-1][:20]
    probs = np.sort(theta_model.beta[k])[::-1][:20]
    top_words = [theta_model.id2word[i] for i in word_ids]
    strings = [f'{prob} * {word}' for prob, word in zip(probs, top_words)]
    print(f"Topic {str(k)}: {', '.join(strings)} \n") 
    #Add a distribution of p(reln|word, topic, top theta role)
    #First, need to know top theta role for this specific topic based on distribution of each theta role for top documents...
    top_documents_indx = np.argsort(theta_model.theta[:, k])[::-1][:top_doc]
    top_theta_ids = top_theta_role_for_topic(k, top_documents_indx)[0:1]
    print('top_theta_ids: ', top_theta_ids)
    for widx in word_ids:
        dic_reln = {reln:0 for reln in theta_model.vocab_relns} #@zhila double check this one.. 
        for t in top_theta_ids:
#             print(widx, k, t)
            dist = compute_reln_distribution_for_topic_words(widx, k, t)
#             print('*** ', dist)
            #merge dist for all the theta role for this topic..sum dist or average dist
            for ki,v in dist.items():
                dic_reln[ki] += v
        #sort based on probability of reln for all the theta role...
        results = sorted(dic_reln.items(), key=lambda x: x[1], reverse=True)
        print('-------- -------- -------- -------- -------- -------- \n')
        print(f"* {str(theta_model.id2word[widx])}: Relns: {', '.join([result[0]+':'+str(result[1]) for result in results[0:3]])} \n")
                


In [ ]:
# dist

In [ ]:
compute_reln_distribution_for_topic_words(1,2,3)

In [ ]:
np.argsort(theta_model.theta[:, 18])[::-1][:5]

In [ ]:
# str(theta_model.id2word[widx])
# widx, k, t
top_theta_role_for_topic(2)

In [ ]:
# theta_model.print_topics_word_reln_distribution()

In [ ]:
for t in range(10):
    reln_ids = np.argsort(theta_model.zeta[t])[::-1][:10]
    probs = np.sort(theta_model.zeta[t])[::-1][:10]
    top_relns = [theta_model.vocab_relns[i] for i in reln_ids]
    strings = [f'{prob} * {reln}' for prob, reln in zip(probs, top_relns)]
    print(f"Theta Role {str(t)}: {', '.join(strings)}\n")

In [ ]:
top_doc=20
'''
print top documents for each topic
'''

for k in range(theta_model.K):
    top_documents_indx = np.argsort(theta_model.theta[:, k])[::-1][:top_doc]
    top_thetas = 5
    print('topic {0} \n'.format(k))
    try:
        for idx in top_documents_indx:
            print("doc {0} - {1} - {2} \n ".format(idx, theta_model.theta[idx, k], theta_model.originaltext[str(idx)]))
        print('------------------------------------------------------------------------')
        #now get the top theta role for this specific document...
        top_thetas_for_doc_indx = np.argsort(theta_model.phi[idx])[::-1][:top_thetas]
        try:
            for t in top_thetas_for_doc_indx:
                print("theta {0} - {1} \n ".format(t, theta_model.phi[idx, t]))
#                         
        except:
            print(top_documents_indx)
    except:
        print("error")

In [ ]:
#testing block to get theta role in each top document?
top_doc=10
top_thetas = 5
for k in range(theta_model.K):
    top_documents_indx = np.argsort(theta_model.theta[:, k])[::-1][:top_doc]
    print('****')
    print('topic {} \n'.format(k))
    try:
        for idx in top_documents_indx:
            print("doc {0} \n ".format(idx))
            print("doc {0} - {1} - {2} \n ".format(idx, theta_model.theta[idx, k], theta_model.originaltext[str(idx)]))

            #now get the top theta role for this specific document...
            top_thetas_for_doc_indx = np.argsort(theta_model.phi[idx])[::-1][:top_thetas]
            try:
                for t in top_thetas_for_doc_indx:
                    print("theta {0} - {1} \n ".format(t, theta_model.phi[idx, t]))
            except:
                print(top_documents_indx)
    except:
        print('error')

In [ ]:
theta_model.theta[0, 4]

In [ ]:
# top_topics=5
# for idx in range(theta_model.D):
#     top_topics_indx = np.argsort(theta_model.theta[idx, :][::-1][:top_topics])
#     print(top_topics_indx)
#     print('document {0} \n'.format(idx))
#     try:
#         for k in top_topics_indx:
#             print("topic {0} - {1} \n ".format(k, theta_model.theta[idx, k]))
#         print('********************')
#     except:
#         print('some error')
# top_thetas = 5
# for idx in range(theta_model.D):
#     top_thetas_indx = np.argsort(theta_model.phi[idx, :][::-1][:top_thetas])
#     print('document {0} \n'.format(idx))
#     try:
#         for t in top_thetas_indx:
#             print("theta {0} - {1} \n ".format(t, theta_model.phi[idx, t]))
#         print('------------------------------------------------------------------------')
#         print('------------------------------------------------------------------------')
#     except:
#         print('error')

In [ ]:
doc_objects['0']['originaltext']

In [ ]:
# for doc_id in doc_objects:
#     print(doc_id)

In [ ]:
# dict = {doc_id: doc_objects[str(doc_id)]['originaltext'] for doc_id in doc_objects}

In [ ]:
# originaltext = list(map(lambda x: id2word.doc2idx(x), text))
# originaltext

In [ ]:
text[0]

In [ ]:
# corpus
# top_documents_indx = np.argsort(theta_model.theta[:, 4])[::-1][:10]
# top_documents_indx

# Top documents for topics

# Top documents for theta roles 

In [ ]:
top_documents_indx = theta_model.print_top_documents_theta()

In [ ]:
top_documents_indx